## Week 3: Tutorial

## Goal: Investigate archaic ancestry in modern humans

### Set your individual

In [ ]:
# REPLACE with your individual
my_individual = 'HG01149'

## Installing requirements

Connect to Github and load the necessary data and tools (runtime: 2min)

In [ ]:
# %%capture
!export LD_LIBARY_PATH=/usr/lib/x86_64-linux-gnu:$LD_LIBRARY_PATH
!apt install libgsl-dev
!ln -s /usr/lib/x86_64-linux-gnu/libgsl.so /usr/lib/x86_64-linux-gnu/libgsl.so.0
!git clone https://github.com/CCB293/Fall-2021 
# download necessary tools
#!apt install autoconf autogen
#!git clone https://github.com/vcftools/vcftools.git
#!cd vcftools && ./autogen.sh && ./configure && make && make install
#!git clone https://github.com/samtools/htslib.git
#!cd htslib && autoheader && autoconf && ./configure && make && make install
!chmod +x Fall-2021/bin/smartpca
!chmod +x Fall-2021/bin/admixture
!chmod +x Fall-2021/bin/tabix
!chmod +x Fall-2021/bin/vcftools
!cd Fall-2021/data/1000G_archaic/ && unzip 1000G_archaic.geno.zip
!echo "Packages installed"

Obtain all necessary data from 1000 Genomes project (runtime: 5min)

In [ ]:
# Download sample information
!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel
# Download vcf file
!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
# Download index file
!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi
#Unpack human ancestral sequence. Useful for polarizing variants in ancestral and derived states
!gunzip Fall-2021/data/1000G_archaic/homo_sapiens_ancestor_17.fa.gz
# Move files to local directory
!mv ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz /content/Fall-2021/data/1000G_archaic/
!mv ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi /content/Fall-2021/data/1000G_archaic/
!echo "Data download completed"

In [ ]:
# load the libraries
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import colorsys
import seaborn as sns
from IPython.display import Image
import os

Define plotting functions

In [ ]:
# define plotting functions

def plot_pcs(pcs, I, J, labels, indivs=None):
    PCI = 'PC{}'.format(I)
    PCJ = 'PC{}'.format(J)
    plt.figure(figsize=(10,10))
    colors = [
        "#7e1e9c", "#15b01a", "#0343df", "#ff81c0","#653700","#e50000","#029386",
        "#f97306", "#96f97b", "#c20078", "#ffff14", "#95d0fc", "#929591",
        "#9a0eea", "#033500", "#00035b","#06c2ac", "#d1b26f","#d1b26f","#d1b26f",
        "#d1b26f","#00ffff","#650021", "#ffb07c","#ff796c","#36013f","#c65102",
        "#000000", "#c1f80a","#4b5d16","#0652ff"]

    print(len(np.unique(labels)))
    for i, p in enumerate(np.unique(labels)):
        pcs_p = pcs[labels==p]
        plt.scatter(pcs_p[PCI], pcs_p[PCJ], label=p, color=colors[i])
    plt.legend(loc=(1.04,0))
    if indivs is not None:
        for individual in indivs:
            pcs_i = pcs[pcs.Sample==individual]
            plt.annotate(individual, [pcs_i[PCI], pcs_i[PCJ]])
            plt.plot(pcs_i[PCI], pcs_i[PCJ], 'o', color='black')
    plt.xlabel(PCI)
    plt.ylabel(PCJ);

def plot_admixture(admixture, population_indices, population_labels):
    plot = plt

    N,K = admixture.shape
    colors = [colorsys.hsv_to_rgb(h,0.9,0.7) for h in np.linspace(0,1,K+1)[:-1]]
    text_color = 'k'
    bg_color = 'w'
    fontsize = 24

    figure = plot.figure(figsize=(5,3))

    xmin = 0.13
    ymin = 0.2
    height = 1.2#0.6
    width = 1.5#0.74
    indiv_width = width/N
    subplot = figure.add_axes([xmin,ymin,width,height])
    [spine.set_linewidth(0.001) for spine in subplot.spines.values()]

    for k in np.arange(K):
        if k:
            bottoms = admixture[:,:k].sum(1)
        else:
            bottoms = np.zeros((N,),dtype=float)

        lefts = np.arange(N)*indiv_width
        subplot.bar(lefts, admixture[:,k], width=indiv_width, bottom=bottoms, facecolor=colors[k], edgecolor=colors[k], linewidth=0.4)

        subplot.axis([0, N*indiv_width, 0, 1])
        subplot.tick_params(axis='both', top=False, right=False, left=False, bottom=False)
        xtick_labels = tuple(map(str,['']*N))
        subplot.set_xticklabels(xtick_labels)
        ytick_labels = tuple(map(str,['']*K))
        subplot.set_yticklabels(ytick_labels)

    for p,popname in enumerate(population_labels):
        indices = np.where(population_indices==p)[0]
        if indices.size>0:
            vline_pos = (indices.max()+1)*indiv_width 
            subplot.axvline(vline_pos, linestyle='-', linewidth=0.2, c='#888888')
            label_position = (xmin+(2*indices.min()+indices.size)*0.5*indiv_width, ymin-0.01)
            figure.text(label_position[0], label_position[1], popname, fontsize=12, color='k', \
                horizontalalignment='right', verticalalignment='top', rotation=70)

    return figure

def plot_admix_individual(admix, individual):
    K = admix.shape[1]
    colors = [colorsys.hsv_to_rgb(h,0.9,0.7) for h in np.linspace(0,1,K+1)[:-1]]
    plt.pie(admix[individuals.individual==individual][0], colors=colors)
    plt.title(individual);

def plot_window_assignment(window_assignment):
    plt.scatter(np.arange(len(window_assignment)), window_assignment, s=8)
    plt.xlabel('window')
    plt.yticks([0,1], ['Human', 'Archaic'])

In [ ]:
# set current directory
%cd Fall-2021/data/1000G_archaic/

#### 1000 Genomes Populations and 3 letter codes. 
For reference, see population_info.csv

In [ ]:
%%writefile population_info.csv
population,description,super_population
CHB,Han Chinese in Beijing China,EAS
JPT,Japanese in Tokyo - Japan,EAS
CHS,Southern Han Chinese,EAS
CDX,Chinese Dai in Xishuangbanna - China,EAS
KHV,Kinh in Ho Chi Minh City - Vietnam,EAS
CEU,Utah Residents (CEPH) with Northern and Western European Ancestry,EUR
TSI,Toscani in Italia,EUR
FIN,Finnish in Finland,EUR
GBR,British in England and Scotland,EUR
IBS,Iberian Population in Spain,EUR
YRI,Yoruba in Ibadan - Nigeria,AFR
LWK,Luhya in Webuye - Kenya,AFR
GWD,Gambian in Western Divisions in the Gambia,AFR
MSL,Mende in Sierra Leone,AFR
ESN,Esan in Nigeria,AFR
ASW,Americans of African Ancestry in SW USA,AFR
ACB,African Caribbeans in Barbados,AFR
MXL,Mexican Ancestry from Los Angeles USA,AMR
PUR,Puerto Ricans from Puerto Rico,AMR
CLM,Colombians from Medellin - Colombia,AMR
PEL,Peruvians from Lima - Peru,AMR
GIH,Gujarati Indian from Houston - Texas,SAS
PJL,Punjabi from Lahore - Pakistan,SAS
BEB,Bengali from Bangladesh,SAS
STU,Sri Lankan Tamil from the UK,SAS
ITU,Indian Telugu from the UK,SAS

In [ ]:
# Read the individual file. For more information about file formats, refer to: https://reich.hms.harvard.edu/software/InputFileFormats 
individuals = pd.read_csv('1000G_archaic.ind', delim_whitespace=True, header=None, names=['individual', 'sex', 'population'])

In [ ]:
population_info = pd.read_csv('population_info.csv')

In [ ]:
individuals = individuals.merge(population_info, on='population', how='left').fillna('Archaic')
individuals.at[2505, 'population'] = 'Archaic'
individuals.at[2504, 'population'] = 'Archaic'

In [ ]:
individuals

**1000 Genomes superpopulations:** <br>
&emsp;&emsp;AFR = Africa<br>
&emsp;&emsp;EUR = Europe<br>
&emsp;&emsp;SAS = South Asia<br>
&emsp;&emsp;EAS = East Asia<br>
&emsp;&emsp;AMR = Americas<br>
&emsp;&emsp;Archaic = Neanderthal/ Denisova

In [ ]:
individuals.population = individuals.population.astype('category')
individuals.super_population = pd.Categorical(
    individuals.super_population, 
    categories=['AFR', 'EUR', 'SAS', 'EAS', 'AMR', 'Archaic'],
    ordered=True)

In [ ]:
# sort individuals by population
individuals = individuals.sort_values(['super_population', 'population'])

In [ ]:
# order individual indexes
order = individuals.index

In [ ]:
# Set my_individual's continental group
my_individual_index = np.where(individuals.individual==my_individual)[0][0]
my_continental_group = individuals.iloc[my_individual_index].super_population

In [ ]:
snps = pd.read_csv('1000G_archaic.snp', delim_whitespace=True, header=None, names=['id', 'chr', 'recomb. rate', 'position', 'allele 1', 'allele 2'])

In [ ]:
genotypes = pd.read_fwf('1000G_archaic.geno', widths=np.ones(2506, dtype=int).tolist(), header=None)

# Principal Component Analysis (PCA)

Patterson, Nick, Alkes L. Price, and David Reich. "Population structure and eigenanalysis." PLoS genetics 2.12 (2006): e190.

https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.0020190

Software documentation: https://github.com/DReichLab/EIG/tree/master/POPGEN

Usage: `!../../bin/smartpca -p 1000G_archaic.smartpca.par`

In [ ]:
# Parameter file (1000G_archaic.smartpca.par)
%%writefile 1000G_archaic.smartpca.par
genotypename: 1000G_archaic.geno     # file with genotype information
snpname:      1000G_archaic.snp      # file with snp information
indivname:    1000G_archaic.ind      # file with individual information   
#poplistname:  1000G_archaic.pop.list # list of pops to include in the run.
outliermode: 2
evecoutname:  1000G_archaic.evec     # output file of eigenvectors.
evaloutname:  1000G_archaic.eval     # output file of all eigenvalues
snpweightoutname: 1000G_archaic.Weightout.txt
phylipname:   1000G_archaic.phyl     # file with Fst values across populations 
numoutevec:   20               # number of PCs to output 
numthreads:   1                # if running interactively, use 1 only

**Run smartpca**

In [ ]:
# %%capture
!../../bin/smartpca -p 1000G_archaic.smartpca.par

Display results - evec file. File contains loadings for each individual on the 20 PCs. <br>https://github.com/DReichLab/EIG/tree/master/POPGEN

In [ ]:
n_pcs = 20
pcs = pd.read_csv('1000G_archaic.evec', skiprows=1, header=None, delim_whitespace=True,
                       names=['Sample']+['PC'+str(i+1) for i in range(n_pcs)]+['population'])
pcs = pcs.merge(population_info, on='population', how='left').fillna('Archaic')
pcs.head()

In [ ]:
# Plot the top two PCs. Population labels = super_population
plot_pcs(pcs, 1, 2, pcs.super_population, [my_individual])
#setting my_individual as an archaic, can see how it clusters relative to humans

In [ ]:
# Plot the top two PCs. Population labels = population
plot_pcs(pcs, 1, 2, pcs.population, [my_individual])

Display eigenvalues. https://github.com/DReichLab/EIG/tree/master/POPGEN

In [ ]:
eigenvalues = np.loadtxt('1000G_archaic.eval')
plt.plot(np.arange(1, 21), eigenvalues[:20])
plt.xticks(np.arange(1, 21));
plt.xlabel('eigenvalue rank')
plt.ylabel('eigenvalue');
#remove these plots? Showed them last time

Display %variance explained

In [ ]:
eigenvalues = np.loadtxt('1000G_archaic.eval')
plt.plot(np.arange(1, 21), 100 * eigenvalues[:20] / eigenvalues.sum())
plt.xticks(np.arange(1, 21));
plt.xlabel('eigenvalue rank')
plt.ylabel('percent variance explained');

# Admixture analysis

Alexander, David H., John Novembre, and Kenneth Lange. "Fast model-based estimation of ancestry in unrelated individuals." Genome research 19.9 (2009): 1655-1664.

https://genome.cshlp.org/content/19/9/1655.full

Software documentation: http://www.genetics.ucla.edu/software/admixture/admixture-manual.pdf

Usage: 

```!../../bin/admixture --cv 1000G_archaic.bed K```

where K = number of clusters

Running ADMIXTURE with K=2 clusters (runtime: ~4m)

In [ ]:
!../../bin/admixture --cv 1000G_archaic.bed 2

Each line in the output of ADMIXTURE contains the probability of an individual belonging to each cluster - not be to confused with ancestry proportion.

In [ ]:
admix = pd.read_csv('1000G_archaic.2.Q', ' ', header=None).values[order]
admix

Admixture plot where each color represents one cluster

In [ ]:
plot_admixture(admix, individuals.super_population.cat.codes, individuals.super_population.cat.categories);

Admixture plot with subcontinental ancestry

In [ ]:
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [ ]:
# Admixture output for Altai Neanderthal. 
plot_admix_individual(admix, 'Altai')

In [ ]:
# Admixture output for Denisova. 
plot_admix_individual(admix, 'Denisova')

For K=3,4,5,6 results have been precomputed, we can just load them. <br>Display output of ADMIXTURE (K=3)

In [ ]:
admix = pd.read_csv('1000G_archaic.3.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

### Stop! Check your understanding
1. What are the (rough) probabilities that an individual with Mexican ancestry belongs to the blue, green, and red clusters? 

In [ ]:
# Admixture output for the current individual (K=3)
plot_admix_individual(admix, my_individual)

In [ ]:
# Admixture output for Altai Neanderthal. 
plot_admix_individual(admix, 'Altai')

In [ ]:
# Admixture output for Denisova. 
plot_admix_individual(admix, 'Denisova')

Display output of ADMIXTURE (K=4)

In [ ]:
admix = pd.read_csv('1000G_archaic.4.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [ ]:
# Admixture output for the current individual (K=4)
plot_admix_individual(admix, 'Altai')

In [ ]:
plot_admix_individual(admix, 'Denisova')

Display output of ADMIXTURE (K=5)

In [ ]:
admix = pd.read_csv('1000G_archaic.5.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [ ]:
# Admixture output for the current individual (K=5)
plot_admix_individual(admix, 'Altai')

In [ ]:
plot_admix_individual(admix, 'Denisova')

Display output of ADMIXTURE (K=6)

In [ ]:
admix = pd.read_csv('1000G_archaic.6.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [ ]:
# Admixture output for the current individual (K=6)
plot_admix_individual(admix, 'Altai')

In [ ]:
plot_admix_individual(admix, 'Denisova')

# Analysis of archaic introgression

Skov, Laurits, et al. "Detecting archaic introgression using an unadmixed outgroup." PLoS genetics 14.9 (2018): e1007641.

https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1007641

Software: https://github.com/LauritsSkov/Introgression-detection

####Key Steps:
###### Estimate the average mutation rate per individual (removing variants observed in outgroup)
`!python2 ../../bin/Estimate_mutationrate.py outgroup_freqfile windowsize window_offset mask_file outputfile

###### Train the HMM to infer the maximum likelihood parameters per individual
`!python2 ../../bin/Train.py infile outprefix model weights_file mutfile`

###### Decode the HMM to infer posterior probability of archaic ancestry in each window
`!python2 ../../bin/Decode.py infile outprefix model weights_file mutfile window_size`

In [ ]:
# Main idea for identifying segments of archaic ancestry
Image(filename='hmm-skov-2.png', height=500)

In [ ]:
# Key steps in analysis
Image(filename='hmm-skov-3.png', height=500)

#### Estimate the average mutation rate per individual (removing variants observed in outgroup) (runtime: 10min)

In [ ]:
# Make a list of outgroups (here, 1000G African populations YRI, ESN and MSL)
!cat ../../../integrated_call_samples_v3.20130502.ALL.panel | grep -E "YRI|ESN|MSL" | cut -f 1 > outgroups.txt

# Estimate the frequency of variants in outgroup
!../../bin/tabix -h ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz -B chr17.bed | ../../bin/vcftools --vcf - --counts --stdout --keep outgroups.txt --remove-indels --min-alleles 2 --max-alleles 2 > chr17.freq

# Estimate the average mutation rate in windows of 1MB with offset of 1000 bp. Choice of 1MB is based on diversity patterns in humans. see http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1007254
!python2 ../../bin/Estimate_mutationrate.py chr17.freq 1000000 1000 chr17.txt chr17.mut

# Estimate SNP density in the assigned individual. Remove variants seen in outgroups.
!../../bin/tabix -fh ALL.chr17.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz -B chr17.bed | ../../bin/vcftools --vcf - --indv {my_individual} --remove-indels --min-alleles 2 --max-alleles 2 --stdout --counts | python2 ../../bin/Filtervariants.py homo_sapiens_ancestor_17.fa chr17.freq 1000 chr17.txt {my_individual}.chr17.observations.txt
!echo "Average mutation rates estimated"

Define initial parameters for the HMM

In [ ]:
%%writefile StartingParameters.hmm
# State names (only used for decoding)
states = ['Human','Archaic']

# Initialization parameters (prob of staring in states)
starting_probabilities = [0.98, 0.02]

# transition matrix
transitions = [[0.9995,0.0005],[0.012,0.98]]

# emission matrix (poisson parameter)
emissions = [0.04, 0.1]

#### Train the HMM to infer the maximum likelihood parameters per individual (runtime: 1min)

In [ ]:
# The model will create two output files. One logfile with the likelihood of the model for each iteration 
# and {my_individual}}_trained.hmm which is the same format as StartingParameters.hmm (just with the parameters that optimize the likelihood).
!python2 ../../bin/Train.py {my_individual}.chr17.observations.txt {my_individual}_trained StartingParameters.hmm chr17.txt chr17.mut

In [ ]:
# Display trained parameters
!cat {my_individual}_trained.hmm

#### Decode the HMM to infer posterior probability of archaic ancestry in each window

In [ ]:
# set of trained parameters that maximize the likelihood to estimate the posterior probability of archaic ancestry in each window
# This will also produce two files. One {my_individual}_decoded.Summary which is like a bedfile and tell you what part of the sequence belong to different states. It also tells you how many snps that are in each segment and what the average posterior probability for being in that segment is. 
# The other output file is {my_individual}_decoded.All_posterior_probs.txt and this is a window by window assignment to each state.
!python2 ../../bin/Decode.py {my_individual}.chr17.observations.txt decoded {my_individual}_trained.hmm chr17.txt chr17.mut 1000

#### Plot the output

In [ ]:
# Inference for every 1000bp window
posterior_probs = pd.read_csv('decoded.All_posterior_probs.txt', '\t')
posterior_probs.head()

In [ ]:
posterior_probs[posterior_probs.observations>0][:4]

The most likeliy assignment of ancestry to the whole chromosome, split into contiguous segments of same ancestry

In [ ]:
decoded_summary = pd.read_csv('decoded.Summary.txt', '\t')
decoded_summary.head()

Obtain the windows ancestry assignment

In [ ]:
window_assignment = (posterior_probs.Mostlikely=='Archaic')

### Stop! Check your understanding
1. How many regions are assigned as 'most likely Archaic'?
2. What is the mean length of 'Archaic' segments on this chromosome?

In [ ]:
# Plot the assignment of the whole chromosome
plot_window_assignment(window_assignment)

In [ ]:
# Plot assignment of the first 5000 windows
plot_window_assignment(window_assignment[40000:50000])

In [ ]:
# Obtain number of observations per window of Human or Archaic ancestry
obs_human = posterior_probs.observations[posterior_probs.Mostlikely=='Human']
obs_archaic = posterior_probs.observations[posterior_probs.Mostlikely=='Archaic']
max_human = obs_human.max()
max_archaic = obs_archaic.max()

In [ ]:
# Plot the number of variants per window of human ancestry
plt.bar(np.arange(max_human+1), [(obs_human==i).mean() for i in range(max_human+1)]);
plt.xticks(np.arange(max_human+1));
plt.xlabel('Number of variants')
plt.ylabel('Proportion')
plt.title('Histogram of number of variants per window of Human ancestry');

In [ ]:
# Plot the number of variants per window of archaic ancestry
plt.bar(np.arange(max_archaic+1), [(obs_archaic==i).mean() for i in range(max_archaic+1)]);
plt.xticks(np.arange(max_archaic+1));
plt.xlabel('Number of variants')
plt.ylabel('Proportion')
plt.title('Histogram of number of variants per window of Archaic ancestry');